In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np

import folium
import networkx as nx

In [216]:
Daegu = folium.Map(location=[35.85,128.6],zoom_start=13)
link_data=pd.read_csv('Daegu_link_1.csv') 
node_data=pd.read_csv('Daegu_node_1.csv') 

In [181]:
link_node=link_data.F_NODE.tolist()+link_data.T_NODE.tolist()
link_node=list(set(link_node))
link_node=pd.DataFrame(link_node,columns=['NODE_ID'])
node_node=node_data[['NODE_ID','X','Y','STNL_REG']]
node_set=pd.merge(node_node,link_node,on='NODE_ID')
front=link_data[['LINK_ID', 'F_NODE']]
tail=link_data[['LINK_ID', 'T_NODE']]
node_node2=node_node.drop('STNL_REG',axis=1)
node_node3=node_node2.rename(columns={'NODE_ID':'F_NODE' , 'X':'F_X', 'Y':'F_Y' })
node_node4=node_node2.rename(columns={'NODE_ID':'T_NODE' , 'X':'T_X', 'Y':'T_Y' })
front2=pd.merge(front,node_node3,on='F_NODE')
tail2=pd.merge(tail,node_node4,on='T_NODE')
node5=pd.merge(front2,tail2, on = 'LINK_ID')

In [182]:
G=nx.DiGraph()
G.add_nodes_from(node_set.NODE_ID.tolist())
for i in range (0,len(node5)):
    G.add_edge(node5.F_NODE[i],node5.T_NODE[i])
    folium.PolyLine([(node5.F_Y[i],node5.F_X[i]),(node5.T_Y[i],node5.T_X[i])],color='black').add_to(Daegu)
BC=nx.betweenness_centrality(G)
CC=nx.closeness_centrality(G)

In [183]:
BC_keys=list(BC.keys())
BC_values=list(BC.values())
CC_keys=list(CC.keys())
CC_values=list(CC.values())

BC1=pd.DataFrame({'NODE_ID':BC_keys,'BC':BC_values})
CC1=pd.DataFrame({'NODE_ID':CC_keys,'CC':CC_values})
BC_CC=pd.merge(BC1,CC1,on='NODE_ID')
node_set=pd.merge(node_set,BC_CC,on='NODE_ID')

In [217]:
layer1=folium.FeatureGroup(name='node')
layer2=folium.FeatureGroup(name='BC')
layer3=folium.FeatureGroup(name='CC')
for i in range (0,len(node_set)):
    folium.CircleMarker(location=[node_set.Y[i],node_set.X[i]],tooltip = node_set.STNL_REG[i],color='yellow',fill=True,fill_color='yellow').add_to(layer1)
    folium.CircleMarker(location=[node_set.Y[i],node_set.X[i]],tooltip = node_set.CC[i],color='pink',fill=True,fill_color='red',radius=5+100*node_set.CC[i]).add_to(layer2)
    folium.CircleMarker(location=[node_set.Y[i],node_set.X[i]],tooltip = node_set.BC[i],color='sky blue',fill=True,fill_color='blue',radius=5+500*node_set.BC[i]).add_to(layer3)
Daegu.add_child(layer1)
Daegu.add_child(layer2)
Daegu.add_child(layer3)
Daegu.add_child(folium.LayerControl())
Daegu.save('Daegu.html')   